# WEB- SCRAPING ON xml file


In [3]:
# XML scrapping for xml sheet 

import os
os.chdir(r"C:\Users\shali\Desktop\DS_Road_Map\9. NLP\NLP WEB SCRAPING\xml_single articles")

import xml.etree.ElementTree as ET

tree = ET.parse("769952.xml") 
root = tree.getroot()

root=ET.tostring(root, encoding='utf8').decode('utf8')

root

import re, string, unicodedata
import nltk

from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

def strip_html(text):
    soup = BeautifulSoup(text, "xml")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub(r'\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text=re.sub('  ','',text)
    return text

sample = denoise_text(root)
print(sample)





0901c7918047d0e2

Orphan Drug Approvals




WebMD, LLC



index



0901c79180555528


News Alert





FDA Grants Orphan Drug Status to Gevokizumab

The FDA has granted orphan drug designation to gevokizumab for the treatment of noninfectious intermediate uveitis, posterior uveitis, or panuveitis, or chronic noninfectious anterior uveitis.




Troy Brown

Journalist

Troy Brown is a freelance writer for Medscape.


Disclosure
Troy Brown has disclosed no relevant financial relationships.


Title





29
08
2012








choroiditis,cyclitis,intermediate uveitis,orphan drugs,pars planitis,posterior uveitis



29
08
2012











August 29, 2012 — The US Food and Drug Administration (FDA) has granted orphan drug status to gevokizumab (Xoma 052, Xoma Corp), a monoclonal antibody that binds strongly to interleukin 1β (IL-1β), for the treatment of noninfectious intermediate uveitis, posterior uveitis, or panuveitis, or chronic noninfectious anterior uveitis.
The Orphan Drug Act of 1983 wa

In [ ]:
# -*- coding: utf-8 -*-
"""XML_Data_Extractor_Visualizer.ipynb

Automatically generated by Colaboratory.

Original file is located at:
    https://colab.research.google.com/drive/1YrQh3p3_5J3v0_3p5_6_7v8w9x0y1z2
"""

# !pip install gradio
# !pip install xmltodict
# !pip install beautifulsoup4

import gradio as gr
import xml.etree.ElementTree as ET
import xmltodict
import json
from bs4 import BeautifulSoup
import re
import pandas as pd
from collections import defaultdict
import plotly.graph_objects as go
import plotly.express as px

def parse_xml(xml_content, input_type):
    """
    Parse XML content from either file upload or text input
    """
    try:
        if input_type == "file":
            # Ensure the file object is read correctly
            if hasattr(xml_content, 'name'):
                with open(xml_content.name, 'r', encoding='utf-8') as f:
                    xml_string = f.read()
            else:
                 return None, None, None, "Invalid file object provided."
        else:
            xml_string = xml_content

        # Parse with ElementTree
        root = ET.fromstring(xml_string)

        # Convert to dict for easier processing
        xml_dict = xmltodict.parse(xml_string)

        return root, xml_dict, xml_string, None
    except Exception as e:
        return None, None, None, str(e)

def extract_xml_structure(xml_content, input_type):
    """
    Extract and display the structure of the XML document
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}"

    # Create a structure visualization
    structure = []

    def traverse(node, depth=0):
        structure.append("  " * depth + f"<{node.tag}>")
        for child in node:
            traverse(child, depth + 1)
        if not list(node):  # If no children, show text content if exists
            if node.text and node.text.strip():
                structure.append("  " * (depth + 1) + f"Text: {node.text.strip()}")

    traverse(root)
    return "\n".join(structure)

def extract_xml_data(xml_content, input_type):
    """
    Extract data from XML and present in a readable format
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}", ""

    # Convert to pretty JSON for display
    json_data = json.dumps(xml_dict, indent=2)

    # Create a simplified table view for common XML structures
    tables = []

    # Try to find repeating elements that might form a table
    def find_tables(node, path=""):
        if len(node) > 0:
            # Check if all children have the same structure
            child_tags = [child.tag for child in node]
            if len(set(child_tags)) == 1 and len(child_tags) > 1:
                # This might be a table-like structure
                columns = set()
                for child in node:
                    for grandchild in child:
                        columns.add(grandchild.tag)

                if columns:
                    # Create a table
                    table_data = []
                    for child in node:
                        row = {}
                        for grandchild in child:
                            row[grandchild.tag] = grandchild.text if grandchild.text else ""
                        table_data.append(row)

                    if table_data:
                        df = pd.DataFrame(table_data)
                        tables.append((f"Table from {path}/{node.tag}", df))

            # Recursively process children
            for child in node:
                find_tables(child, f"{path}/{node.tag}")

    find_tables(root)

    # Format tables for display
    tables_html = "<h2>Extracted Tables</h2>"
    if tables:
        for name, df in tables:
            tables_html += f"<h3>{name}</h3>"
            tables_html += df.to_html()
    else:
        tables_html += "<p>No table-like structures detected</p>"

    return json_data, tables_html

def visualize_xml(xml_content, input_type):
    """
    Create visualizations from XML data
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}"

    # Count elements by tag
    tag_count = defaultdict(int)

    def count_tags(node):
        tag_count[node.tag] += 1
        for child in node:
            count_tags(child)

    count_tags(root)

    # Create a bar chart of tag frequencies
    if tag_count:
        fig = px.bar(x=list(tag_count.keys()), y=list(tag_count.values()),
                     labels={'x': 'XML Tags', 'y': 'Count'},
                     title='Frequency of XML Tags')
        return fig
    else:
        return "No data available for visualization"

def clean_xml(xml_content, input_type):
    """
    Clean and format XML content
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}"

    # Pretty print the XML
    soup = BeautifulSoup(xml_string, 'xml')
    pretty_xml = soup.prettify()

    return pretty_xml

# Create the Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="XML Data Extractor and Visualizer") as demo:
    gr.Markdown("""
    # XML Data Extractor and Visualizer
    Upload an XML file or paste XML content to explore its structure, extract data, and create visualizations.
    """)

    with gr.Tab("XML Input"):
        input_type = gr.Radio(["file", "text"], value="file", label="Input Type")
        xml_input = gr.File(label="Upload XML File", file_types=[".xml"])
        xml_text = gr.Textbox(label="Or Paste XML Content", lines=10, visible=False)

        def toggle_input(choice):
            if choice == "file":
                return gr.File(visible=True), gr.Textbox(visible=False)
            else:
                return gr.File(visible=False), gr.Textbox(visible=True)

        input_type.change(toggle_input, input_type, [xml_input, xml_text])

    with gr.Tab("XML Structure"):
        structure_btn = gr.Button("Extract Structure")
        structure_output = gr.Textbox(label="XML Structure", lines=15)

    with gr.Tab("Data Extraction"):
        extract_btn = gr.Button("Extract Data")
        with gr.Row():
            json_output = gr.Textbox(label="JSON Representation", lines=15)
            table_output = gr.HTML(label="Tabular Data")

    with gr.Tab("Visualization"):
        viz_btn = gr.Button("Create Visualization")
        viz_output = gr.Plot(label="Tag Frequency Visualization")

    with gr.Tab("XML Cleaner"):
        clean_btn = gr.Button("Clean and Format XML")
        clean_output = gr.Code(label="Formatted XML", lines=15)

    # Set up event handlers
    structure_btn.click(
        extract_xml_structure,
        inputs=[xml_input, input_type],
        outputs=structure_output
    )

    extract_btn.click(
        extract_xml_data,
        inputs=[xml_input, input_type],
        outputs=[json_output, table_output]
    )

    viz_btn.click(
        visualize_xml,
        inputs=[xml_input, input_type],
        outputs=viz_output
    )

    clean_btn.click(
        clean_xml,
        inputs=[xml_input, input_type],
        outputs=clean_output
    )

# Launch the interface
if __name__ == "__main__":
    demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# -*- coding: utf-8 -*-
"""XML_Data_Extractor_Visualizer.ipynb

Automatically generated by Colaboratory.

Original file is located at:
    https://colab.research.google.com/drive/1YrQh3p3_5J3v0_3p5_6_7v8w9x0y1z2
"""

!pip install gradio
!pip install xmltodict
!pip install beautifulsoup4
!pip install plotly

import gradio as gr
import xml.etree.ElementTree as ET
import xmltodict
import json
from bs4 import BeautifulSoup
import re
import pandas as pd
from collections import defaultdict
import plotly.graph_objects as go
import plotly.express as px
import tempfile
import os

def parse_xml(xml_content, input_type):
    """
    Parse XML content from either file upload or text input
    """
    try:
        if input_type == "file":
            if xml_content is None:
                return None, None, None, "No file uploaded"
            with open(xml_content.name, 'r', encoding='utf-8') as f:
                xml_string = f.read()
        else:
            xml_string = xml_content
            if not xml_string.strip():
                return None, None, None, "No XML content provided"

        # Parse with ElementTree
        root = ET.fromstring(xml_string)

        # Convert to dict for easier processing
        xml_dict = xmltodict.parse(xml_string)

        return root, xml_dict, xml_string, None
    except Exception as e:
        return None, None, None, str(e)

def extract_xml_structure(xml_content, input_type):
    """
    Extract and display the structure of the XML document
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}"

    # Create a structure visualization
    structure = []

    def traverse(node, depth=0):
        structure.append("  " * depth + f"<{node.tag}>")
        # Add attributes if any
        if node.attrib:
            for attr, value in node.attrib.items():
                structure.append("  " * (depth + 1) + f"@{attr}: {value}")
        for child in node:
            traverse(child, depth + 1)
        if not list(node):  # If no children, show text content if exists
            if node.text and node.text.strip():
                structure.append("  " * (depth + 1) + f"Text: {node.text.strip()}")
        structure.append("  " * depth + f"</{node.tag}>")

    traverse(root)
    return "\n".join(structure)

def extract_xml_data(xml_content, input_type):
    """
    Extract data from XML and present in a readable format
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}", "<p>Error parsing XML</p>"

    # Convert to pretty JSON for display
    try:
        json_data = json.dumps(xml_dict, indent=2)
    except:
        json_data = "Could not convert to JSON format"

    # Extract all data from XML in a structured way
    extracted_data = []

    def extract_elements(node, path=""):
        current_path = f"{path}/{node.tag}" if path else node.tag

        # Extract element data
        element_data = {
            "path": current_path,
            "tag": node.tag,
            "attributes": dict(node.attrib),
            "text": node.text.strip() if node.text and node.text.strip() else None,
            "children": len(list(node))
        }
        extracted_data.append(element_data)

        # Process children
        for child in node:
            extract_elements(child, current_path)

    extract_elements(root)

    # Create a DataFrame for display
    df = pd.DataFrame(extracted_data)

    # Create HTML table
    if not df.empty:
        table_html = df.to_html(classes='table table-striped', index=False, escape=False)
    else:
        table_html = "<p>No data could be extracted from the XML</p>"

    return json_data, table_html

def extract_xml_data_as_table(xml_content, input_type):
    """
    Extract data from XML and present in a tabular format
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}", "<p>Error parsing XML</p>"

    # Find repeating elements that could form a table
    tables = []

    def find_table_data(node, path=""):
        # If this node has children and all children have the same tag, it might be a table
        children = list(node)
        if children:
            child_tags = [child.tag for child in children]
            if len(set(child_tags)) == 1 and len(children) > 1:
                # This looks like a table row structure
                table_rows = []
                for child in children:
                    row = {"_row_type": child.tag}
                    for grandchild in child:
                        row[grandchild.tag] = grandchild.text.strip() if grandchild.text and grandchild.text.strip() else ""
                    table_rows.append(row)

                if table_rows:
                    tables.append({
                        "name": f"{path}/{node.tag}",
                        "data": table_rows
                    })

        # Recursively process children
        for child in children:
            find_table_data(child, f"{path}/{node.tag}")

    find_table_data(root)

    # Generate HTML for tables
    tables_html = "<h2>Extracted Data Tables</h2>"

    if tables:
        for i, table in enumerate(tables):
            df = pd.DataFrame(table["data"])
            tables_html += f"<h3>Table {i+1}: {table['name']}</h3>"
            tables_html += df.to_html(classes='table table-striped', index=False)
    else:
        tables_html += "<p>No table-like structures found. Showing all elements:</p>"

        # Fallback: show all elements in a table
        all_data = []

        def extract_all_elements(node, path=""):
            current_path = f"{path}/{node.tag}" if path else node.tag
            element_data = {
                "Path": current_path,
                "Tag": node.tag,
                "Attributes": str(node.attrib),
                "Text": node.text.strip() if node.text and node.text.strip() else "",
                "Children": len(list(node))
            }
            all_data.append(element_data)

            for child in node:
                extract_all_elements(child, current_path)

        extract_all_elements(root)
        df = pd.DataFrame(all_data)
        tables_html += df.to_html(classes='table table-striped', index=False)

    return tables_html

def visualize_xml(xml_content, input_type):
    """
    Create visualizations from XML data
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}"

    # Count elements by tag
    tag_count = defaultdict(int)

    def count_tags(node):
        tag_count[node.tag] += 1
        for child in node:
            count_tags(child)

    count_tags(root)

    # Create a bar chart of tag frequencies
    if tag_count:
        fig = px.bar(x=list(tag_count.keys()), y=list(tag_count.values()),
                     labels={'x': 'XML Tags', 'y': 'Count'},
                     title='Frequency of XML Tags')
        return fig
    else:
        return "No data available for visualization"

def clean_xml(xml_content, input_type):
    """
    Clean and format XML content
    """
    if input_type == "file":
        if xml_content is None:
            return "No file uploaded"
        with open(xml_content.name, 'r', encoding='utf-8') as f:
            xml_string = f.read()
    else:
        xml_string = xml_content
        if not xml_string.strip():
            return "No XML content provided"

    # Pretty print the XML
    try:
        soup = BeautifulSoup(xml_string, 'xml')
        pretty_xml = soup.prettify()
        return pretty_xml
    except Exception as e:
        return f"Error formatting XML: {str(e)}"

# Create the Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="XML Data Extractor and Visualizer") as demo:
    gr.Markdown("""
    # 🔍 XML Data Extractor and Visualizer
    Upload an XML file or paste XML content to explore its structure, extract data, and create visualizations.
    """)

    with gr.Row():
        with gr.Column():
            input_type = gr.Radio(["file", "text"], value="file", label="Input Type")
            xml_input = gr.File(label="Upload XML File", file_types=[".xml"])
            xml_text = gr.Textbox(label="Or Paste XML Content", lines=10, visible=False)

            def toggle_input(choice):
                if choice == "file":
                    return gr.File(visible=True), gr.Textbox(visible=False)
                else:
                    return gr.File(visible=False), gr.Textbox(visible=True)

            input_type.change(toggle_input, input_type, [xml_input, xml_text])

        with gr.Column():
            gr.Markdown("""
            ### 📊 What you can do:
            - **View Structure**: See the hierarchical structure of your XML
            - **Extract Data**: Convert XML to JSON and detect table structures
            - **Visualize**: Create charts showing tag frequencies
            - **Clean XML**: Format and prettify your XML code

            ### 🚀 Try with this sample XML:
            ```xml
            <catalog>
              <book id="101">
                <author>John Doe</author>
                <title>XML Guide</title>
                <price>29.99</price>
              </book>
              <book id="102">
                <author>Jane Smith</author>
                <title>Python Programming</title>
                <price>39.99</price>
              </book>
            </catalog>
            ```
            """)

    with gr.Tab("📁 XML Structure"):
        gr.Markdown("### XML Document Structure")
        structure_btn = gr.Button("Extract Structure", variant="primary")
        structure_output = gr.Code(label="XML Structure", language="markdown", lines=15)

    with gr.Tab("📊 Data Extraction"):
        gr.Markdown("### Extracted Data from XML")
        extract_btn = gr.Button("Extract Data", variant="primary")
        with gr.Row():
            json_output = gr.Code(label="JSON Representation", language="json", lines=15)
            table_output = gr.HTML(label="Tabular Data")

    with gr.Tab("📈 Visualization"):
        gr.Markdown("### XML Data Visualization")
        viz_btn = gr.Button("Create Visualization", variant="primary")
        viz_output = gr.Plot(label="Tag Frequency Visualization")

    with gr.Tab("✨ XML Cleaner"):
        gr.Markdown("### Clean and Format XML")
        clean_btn = gr.Button("Clean and Format XML", variant="primary")
        clean_output = gr.Code(label="Formatted XML", language="html", lines=15)

    # Set up event handlers
    structure_btn.click(
        extract_xml_structure,
        inputs=[xml_input, input_type],
        outputs=structure_output
    )

    extract_btn.click(
        extract_xml_data,
        inputs=[xml_input, input_type],
        outputs=[json_output, table_output]
    )

    viz_btn.click(
        visualize_xml,
        inputs=[xml_input, input_type],
        outputs=viz_output
    )

    clean_btn.click(
        clean_xml,
        inputs=[xml_input, input_type],
        outputs=clean_output
    )

# For Google Colab, we need to handle the launch differently
def launch_in_colab():
    # Create a public link
    demo.launch(share=True, debug=True)

# Launch the interface
if __name__ == "__main__":
    # This will work in both Colab and local environments
    try:
        import google.colab
        IN_COLAB = True
    except:
        IN_COLAB = False

    if IN_COLAB:
        launch_in_colab()
    else:
        demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://56bcbbe02904c0a8c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://56bcbbe02904c0a8c1.gradio.live


In [16]:
# -*- coding: utf-8 -*-
"""XML_Data_Extractor_Visualizer.ipynb

Automatically generated by Colaboratory.

Original file is located at:
    https://colab.research.google.com/drive/1YrQh3p3_5J3v0_3p5_6_7v8w9x0y1z2
"""

!pip install gradio
!pip install xmltodict
!pip install beautifulsoup4
!pip install plotly

import gradio as gr
import xml.etree.ElementTree as ET
import xmltodict
import json
from bs4 import BeautifulSoup
import re
import pandas as pd
from collections import defaultdict
import plotly.graph_objects as go
import plotly.express as px
import tempfile
import os

def parse_xml(xml_content, input_type):
    """
    Parse XML content from either file upload or text input
    """
    try:
        if input_type == "file":
            if xml_content is None:
                return None, None, None, "No file uploaded"
            with open(xml_content.name, 'r', encoding='utf-8') as f:
                xml_string = f.read()
        else:
            xml_string = xml_content
            if not xml_string.strip():
                return None, None, None, "No XML content provided"

        # Parse with ElementTree
        root = ET.fromstring(xml_string)

        # Convert to dict for easier processing
        xml_dict = xmltodict.parse(xml_string)

        return root, xml_dict, xml_string, None
    except Exception as e:
        return None, None, None, str(e)

def extract_xml_structure(xml_content, input_type):
    """
    Extract and display the structure of the XML document
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}"

    # Create a structure visualization
    structure = []

    def traverse(node, depth=0):
        structure.append("  " * depth + f"<{node.tag}>")
        # Add attributes if any
        if node.attrib:
            for attr, value in node.attrib.items():
                structure.append("  " * (depth + 1) + f"@{attr}: {value}")
        for child in node:
            traverse(child, depth + 1)
        if not list(node):  # If no children, show text content if exists
            if node.text and node.text.strip():
                structure.append("  " * (depth + 1) + f"Text: {node.text.strip()}")
        structure.append("  " * depth + f"</{node.tag}>")

    traverse(root)
    return "\n".join(structure)

def extract_xml_data(xml_content, input_type):
    """
    Extract data from XML and present in a readable format
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}", "<p>Error parsing XML</p>"

    # Convert to pretty JSON for display
    try:
        json_data = json.dumps(xml_dict, indent=2)
    except:
        json_data = "Could not convert to JSON format"

    # Extract all data from XML in a structured way
    extracted_data = []

    def extract_elements(node, path=""):
        current_path = f"{path}/{node.tag}" if path else node.tag

        # Extract element data
        element_data = {
            "path": current_path,
            "tag": node.tag,
            "attributes": dict(node.attrib),
            "text": node.text.strip() if node.text and node.text.strip() else None,
            "children": len(list(node))
        }
        extracted_data.append(element_data)

        # Process children
        for child in node:
            extract_elements(child, current_path)

    extract_elements(root)

    # Create a DataFrame for display
    df = pd.DataFrame(extracted_data)

    # Create HTML table
    if not df.empty:
        table_html = df.to_html(classes='table table-striped', index=False, escape=False)
    else:
        table_html = "<p>No data could be extracted from the XML</p>"

    return json_data, table_html

def generate_insights(xml_content, input_type):
    """
    Generate meaningful insights about the XML structure and content
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"<p style='color: red;'>Error parsing XML: {error}</p>"

    insights_html = "<div style='font-family: Arial, sans-serif;'>"
    insights_html += "<h2 style='color: #2E86AB;'>📊 XML Insights & Analysis</h2>"

    # Basic statistics
    total_elements = 0
    elements_with_attributes = 0
    elements_with_text = 0
    max_depth = 0
    tag_counts = defaultdict(int)
    attribute_counts = defaultdict(int)

    def analyze_node(node, depth=0):
        nonlocal total_elements, elements_with_attributes, elements_with_text, max_depth
        total_elements += 1
        tag_counts[node.tag] += 1
        max_depth = max(max_depth, depth)

        if node.attrib:
            elements_with_attributes += 1
            for attr in node.attrib:
                attribute_counts[attr] += 1

        if node.text and node.text.strip():
            elements_with_text += 1

        for child in node:
            analyze_node(child, depth + 1)

    analyze_node(root)

    # Generate insights
    insights_html += f"<h3 style='color: #A23B72;'>📈 Basic Statistics</h3>"
    insights_html += f"<ul>"
    insights_html += f"<li>Total Elements: <b>{total_elements}</b></li>"
    insights_html += f"<li>Maximum Depth: <b>{max_depth}</b> levels</li>"
    insights_html += f"<li>Elements with Attributes: <b>{elements_with_attributes}</b> ({elements_with_attributes/total_elements*100:.1f}%)</li>"
    insights_html += f"<li>Elements with Text Content: <b>{elements_with_text}</b> ({elements_with_text/total_elements*100:.1f}%)</li>"
    insights_html += f"</ul>"

    # Tag frequency analysis
    insights_html += f"<h3 style='color: #A23B72;'>🏷️ Tag Frequency</h3>"
    if tag_counts:
        insights_html += f"<p>Unique Tags: <b>{len(tag_counts)}</b></p>"
        insights_html += "<table style='width: 100%; border-collapse: collapse;'><tr><th style='border: 1px solid #ddd; padding: 8px; background-color: #f2f2f2;'>Tag</th><th style='border: 1px solid #ddd; padding: 8px; background-color: #f2f2f2;'>Count</th><th style='border: 1px solid #ddd; padding: 8px; background-color: #f2f2f2;'>Percentage</th></tr>"

        for tag, count in sorted(tag_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
            percentage = (count / total_elements) * 100
            insights_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px;'>{tag}</td><td style='border: 1px solid #ddd; padding: 8px;'>{count}</td><td style='border: 1px solid #ddd; padding: 8px;'>{percentage:.1f}%</td></tr>"

        insights_html += "</table>"

        if len(tag_counts) > 10:
            insights_html += f"<p>... and {len(tag_counts) - 10} more tags</p>"

    # Attribute analysis
    if attribute_counts:
        insights_html += f"<h3 style='color: #A23B72;'>🔗 Attributes</h3>"
        insights_html += f"<p>Unique Attributes: <b>{len(attribute_counts)}</b></p>"
        insights_html += "<table style='width: 100%; border-collapse: collapse;'><tr><th style='border: 1px solid #ddd; padding: 8px; background-color: #f2f2f2;'>Attribute</th><th style='border: 1px solid #ddd; padding: 8px; background-color: #f2f2f2;'>Count</th></tr>"

        for attr, count in sorted(attribute_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
            insights_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px;'>{attr}</td><td style='border: 1px solid #ddd; padding: 8px;'>{count}</td></tr>"

        insights_html += "</table>"

        if len(attribute_counts) > 10:
            insights_html += f"<p>... and {len(attribute_counts) - 10} more attributes</p>"

    # Structural insights
    insights_html += f"<h3 style='color: #A23B72;'>🏗️ Structural Insights</h3>"

    # Check for potential data tables
    potential_tables = []

    def find_potential_tables(node, path=""):
        children = list(node)
        if children:
            child_tags = [child.tag for child in children]
            if len(set(child_tags)) == 1 and len(children) > 1:
                # This looks like a table structure
                sample_child = children[0]
                grandchild_count = len(list(sample_child))

                potential_tables.append({
                    "path": f"{path}/{node.tag}",
                    "row_tag": child_tags[0],
                    "row_count": len(children),
                    "column_count": grandchild_count
                })

        for child in children:
            find_potential_tables(child, f"{path}/{node.tag}")

    find_potential_tables(root)

    if potential_tables:
        insights_html += f"<p>Found <b>{len(potential_tables)}</b> potential data table(s):</p>"
        insights_html += "<ul>"
        for table in potential_tables:
            insights_html += f"<li>Location: <code>{table['path']}</code> - {table['row_count']} rows × {table['column_count']} columns</li>"
        insights_html += "</ul>"
    else:
        insights_html += "<p>No obvious table structures detected. This might be a document-oriented XML.</p>"

    # Data quality insights
    insights_html += f"<h3 style='color: #A23B72;'>✅ Data Quality Assessment</h3>"

    empty_elements = total_elements - elements_with_text
    empty_percentage = (empty_elements / total_elements) * 100

    if empty_percentage > 50:
        insights_html += f"<p style='color: #E56B70;'>⚠️ High percentage of empty elements ({empty_percentage:.1f}%). Consider if this is expected for your data model.</p>"
    else:
        insights_html += f"<p>Empty elements: {empty_percentage:.1f}% - This is within normal range.</p>"

    if elements_with_attributes == 0:
        insights_html += f"<p>No attributes found in the XML. This is a simple data structure.</p>"

    # Recommendations
    insights_html += f"<h3 style='color: #A23B72;'>💡 Recommendations</h3>"
    insights_html += "<ul>"

    if max_depth > 5:
        insights_html += "<li>Deeply nested structure detected. Consider simplifying the hierarchy for better performance.</li>"

    if len(tag_counts) > 20:
        insights_html += "<li>Many different tags detected. This might be a complex data model.</li>"

    if potential_tables:
        insights_html += "<li>Table-like structures detected. You might want to convert these to CSV or database tables.</li>"

    insights_html += "<li>For better visualization, consider using the 'Data Extraction' tab to view the data in tabular format.</li>"
    insights_html += "</ul>"

    insights_html += "</div>"

    return insights_html

def visualize_xml(xml_content, input_type):
    """
    Create visualizations from XML data
    """
    root, xml_dict, xml_string, error = parse_xml(xml_content, input_type)

    if error:
        return f"Error parsing XML: {error}"

    # Count elements by tag
    tag_count = defaultdict(int)

    def count_tags(node):
        tag_count[node.tag] += 1
        for child in node:
            count_tags(child)

    count_tags(root)

    # Create a bar chart of tag frequencies
    if tag_count:
        fig = px.bar(x=list(tag_count.keys()), y=list(tag_count.values()),
                     labels={'x': 'XML Tags', 'y': 'Count'},
                     title='Frequency of XML Tags')
        return fig
    else:
        return "No data available for visualization"

def clean_xml(xml_content, input_type):
    """
    Clean and format XML content
    """
    if input_type == "file":
        if xml_content is None:
            return "No file uploaded"
        with open(xml_content.name, 'r', encoding='utf-8') as f:
            xml_string = f.read()
    else:
        xml_string = xml_content
        if not xml_string.strip():
            return "No XML content provided"

    # Pretty print the XML
    try:
        soup = BeautifulSoup(xml_string, 'xml')
        pretty_xml = soup.prettify()
        return pretty_xml
    except Exception as e:
        return f"Error formatting XML: {str(e)}"

# Create the Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="XML Data Extractor and Visualizer") as demo:
    gr.Markdown("""
    # 🔍 XML Data Extractor and Visualizer
    Upload an XML file or paste XML content to explore its structure, extract data, and create visualizations.
    """)

    with gr.Row():
        with gr.Column():
            input_type = gr.Radio(["file", "text"], value="file", label="Input Type")
            xml_input = gr.File(label="Upload XML File", file_types=[".xml"])
            xml_text = gr.Textbox(label="Or Paste XML Content", lines=10, visible=False)

            def toggle_input(choice):
                if choice == "file":
                    return gr.File(visible=True), gr.Textbox(visible=False)
                else:
                    return gr.File(visible=False), gr.Textbox(visible=True)

            input_type.change(toggle_input, input_type, [xml_input, xml_text])

        with gr.Column():
            gr.Markdown("""
            ### 📊 What you can do:
            - **View Structure**: See the hierarchical structure of your XML
            - **Extract Data**: Convert XML to JSON and detect table structures
            - **Generate Insights**: Get meaningful analysis of your XML content
            - **Visualize**: Create charts showing tag frequencies
            - **Clean XML**: Format and prettify your XML code

            ### 🚀 Try with this sample XML:
            ```xml
            <catalog>
              <book id="101">
                <author>John Doe</author>
                <title>XML Guide</title>
                <price>29.99</price>
              </book>
              <book id="102">
                <author>Jane Smith</author>
                <title>Python Programming</title>
                <price>39.99</price>
              </book>
            </catalog>
            ```
            """)

    with gr.Tab("📁 XML Structure"):
        gr.Markdown("### XML Document Structure")
        structure_btn = gr.Button("Extract Structure", variant="primary")
        structure_output = gr.Code(label="XML Structure", language="markdown", lines=15)

    with gr.Tab("📊 Data Extraction"):
        gr.Markdown("### Extracted Data from XML")
        extract_btn = gr.Button("Extract Data", variant="primary")
        with gr.Row():
            json_output = gr.Code(label="JSON Representation", language="json", lines=15)
            table_output = gr.HTML(label="Tabular Data")

    with gr.Tab("🔍 Insights"):
        gr.Markdown("### XML Insights & Analysis")
        insights_btn = gr.Button("Generate Insights", variant="primary")
        insights_output = gr.HTML(label="XML Insights")

    with gr.Tab("📈 Visualization"):
        gr.Markdown("### XML Data Visualization")
        viz_btn = gr.Button("Create Visualization", variant="primary")
        viz_output = gr.Plot(label="Tag Frequency Visualization")

    with gr.Tab("✨ XML Cleaner"):
        gr.Markdown("### Clean and Format XML")
        clean_btn = gr.Button("Clean and Format XML", variant="primary")
        clean_output = gr.Code(label="Formatted XML", language="html", lines=15)

    # Set up event handlers
    structure_btn.click(
        extract_xml_structure,
        inputs=[xml_input, input_type],
        outputs=structure_output
    )

    extract_btn.click(
        extract_xml_data,
        inputs=[xml_input, input_type],
        outputs=[json_output, table_output]
    )

    insights_btn.click(
        generate_insights,
        inputs=[xml_input, input_type],
        outputs=insights_output
    )

    viz_btn.click(
        visualize_xml,
        inputs=[xml_input, input_type],
        outputs=viz_output
    )

    clean_btn.click(
        clean_xml,
        inputs=[xml_input, input_type],
        outputs=clean_output
    )

# For Google Colab, we need to handle the launch differently
def launch_in_colab():
    # Create a public link
    demo.launch(share=True, debug=True)

# Launch the interface
if __name__ == "__main__":
    # This will work in both Colab and local environments
    try:
        import google.colab
        IN_COLAB = True
    except:
        IN_COLAB = False

    if IN_COLAB:
        launch_in_colab()
    else:
        demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7b180ba37e00500d9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7b180ba37e00500d9a.gradio.live
